In [2]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import Operator
import numpy as np

In [ ]:
def create_he_layer(circ, x, scaling_params, variational_params, wires, embed_data):
    n = len(wires)
    if embed_data:
        for i in range(n):
            circ.rx(scaling_params[i] * x[i], wires[i])
    for i in range(n):
        circ.ry(variational_params[i], wires[i])
    for i in range(n):
        circ.rz(variational_params[i + n], wires[i])
    if n == 2:
        circ.cz(wires[0], wires[1])
    else:
        for i in range(n):
            circ.cz(wires[i], wires[(i + 1) % n])


def create_qkhe_circuit(x1, x2, weights, wires, layers, embed_data):
    circ = QuantumCircuit(len(wires))

    for l in range(layers):
        create_he_layer(circ, x1, weights['input_scaling'][l], weights['variational'][l], wires, embed_data and l == 0)
    for l in reversed(range(layers)):
        create_he_layer(circ, x2, weights['input_scaling'][l], weights['variational'][l], wires, embed_data and l == 0)
        circ = circ.inverse()  

    return circ


In [4]:
def compute_kernel_value(circ):
    psi = Statevector.from_instruction(circ)
    return np.real(np.vdot(psi.data, psi.data))

def compute_projected_overlap(circ, projector, wires):
    psi = Statevector.from_instruction(circ)
    projector_op = Operator(projector)
    return np.real(psi.expectation_value(projector_op))

In [10]:
num_qubits = 4
wires = list(range(num_qubits))
layers = 2
x1 = np.random.rand(num_qubits)
x2 = np.random.rand(num_qubits)

weights = {
    "input_scaling": [np.ones(num_qubits) for _ in range(layers)],
    "variational": [np.ones(2 * num_qubits) for _ in range(layers)]
}

circ = create_qkhe_circuit(x1, x2, weights, wires, layers, embed_data=True)
val = compute_kernel_value(circ)
print("X1:", x1)
print("X2:", x2)
print("Kernel Value:", val)


X1: [0.00250193 0.90875182 0.2856814  0.24251118]
X2: [0.09197363 0.77850005 0.99049507 0.14792868]
Kernel Value: 1.0000000000000016


In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from concurrent.futures import ThreadPoolExecutor

# Setup
NUM_QUBITS = 4
LAYERS = 2
NUM_DATAPOINTS = 1000
WIRES = list(range(NUM_QUBITS))

# Generate 30 data points
data = [np.random.rand(NUM_QUBITS) for _ in range(NUM_DATAPOINTS)]

# Dummy weights (replace if needed)
weights = {
    "input_scaling": [np.ones(NUM_QUBITS) for _ in range(LAYERS)],
    "variational": [np.ones(2 * NUM_QUBITS) for _ in range(LAYERS)]
}

# HE Layer
def create_he_layer(circ, x, scaling, var, wires, embed):
    if embed:
        for i in wires:
            circ.rx(scaling[i] * x[i], i)
    for i in wires:
        circ.ry(var[i], i)
    for i in wires:
        circ.rz(var[i + len(wires)], i)
    for i in range(len(wires)):
        circ.cz(wires[i], wires[(i + 1) % len(wires)])

# Circuit builder
def create_qkhe_circuit(x1, x2, weights, wires, layers, embed_data):
    circ = QuantumCircuit(len(wires))
    for l in range(layers):
        create_he_layer(circ, x1, weights["input_scaling"][l], weights["variational"][l], wires, embed_data and l == 0)
    adjoint_circ = QuantumCircuit(len(wires))
    for l in reversed(range(layers)):
        create_he_layer(adjoint_circ, x2, weights["input_scaling"][l], weights["variational"][l], wires, embed_data and l == 0)
    circ.compose(adjoint_circ.inverse(), inplace=True)
    return circ

# Kernel evaluator
def compute_kernel(i, j):
    x1 = data[i]
    x2 = data[j]
    circ = create_qkhe_circuit(x1, x2, weights, WIRES, LAYERS, embed_data=True)
    state = Statevector.from_instruction(circ)
    val = np.real(np.vdot(state.data, state.data))  # ⟨ψ(x_j)|ψ(x_i)⟩
    return (i, j, val)

# Compute full Gram matrix using threading
gram_matrix = np.zeros((NUM_DATAPOINTS, NUM_DATAPOINTS))

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(compute_kernel, i, j) for i in range(NUM_DATAPOINTS) for j in range(NUM_DATAPOINTS)]

    for future in futures:
        i, j, val = future.result()
        gram_matrix[i, j] = val

print("30x30 Quantum Kernel Gram Matrix:")
print(gram_matrix)